In [47]:
import pandas as pd
import numpy as np


In [48]:
df=pd.read_csv('CarPrice_Assignment.csv')

In [49]:
df.head()

,car_ID,symboling,CarName,fueltype,aspiration,doornumber,carbody,drivewheel,enginelocation,wheelbase,...,enginesize,fuelsystem,boreratio,stroke,compressionratio,horsepower,peakrpm,citympg,highwaympg,price
0,1,3,alfa-romero giulia,gas,std,two,convertible,rwd,front,88.6,...,130,mpfi,3.47,2.68,9.0,111,5000,21,27,13495.0
1,2,3,alfa-romero stelvio,gas,std,two,convertible,rwd,front,88.6,...,130,mpfi,3.47,2.68,9.0,111,5000,21,27,16500.0
2,3,1,alfa-romero Quadrifoglio,gas,std,two,hatchback,rwd,front,94.5,...,152,mpfi,2.68,3.47,9.0,154,5000,19,26,16500.0
3,4,2,audi 100 ls,gas,std,four,sedan,fwd,front,99.8,...,109,mpfi,3.19,3.40,10.0,102,5500,24,30,13950.0
4,5,2,audi 100ls,gas,std,four,sedan,4wd,front,99.4,...,136,mpfi,3.19,3.40,8.0,115,5500,18,22,17450.0


In [50]:
x=df.iloc[:,:-1]

In [51]:
y=df.iloc[:,-1]

In [52]:
#handling missng data

In [53]:

from sklearn.impute import SimpleImputer
imp_mean=SimpleImputer(missing_values=np.nan,strategy='mean')

In [54]:
numeric_columns=list(np.where((x.dtypes==np.int64) | (x.dtypes==np.float64))[0])

In [55]:
imp_mean.fit(x.iloc[:,numeric_columns])

SimpleImputer()

In [56]:
x.iloc[:,numeric_columns]=imp_mean.transform(x.iloc[:,numeric_columns])

In [57]:
string_columns=list(np.where((x.dtypes!=np.int64) | (x.dtypes!=np.float64))[0])

In [58]:
imp_mean=SimpleImputer(missing_values=np.nan,strategy='most_frequent')

In [59]:
imp_mean.fit(x.iloc[:,string_columns])

SimpleImputer(strategy='most_frequent')

In [60]:
x.iloc[:,string_columns]=imp_mean.transform(x.iloc[:,string_columns])

In [61]:
x.drop('car_ID',axis=1,inplace=True)

In [62]:
from sklearn import preprocessing
le=preprocessing.LabelEncoder()
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

In [63]:
string_columns=list(np.where((x.dtypes!=np.int64) | (x.dtypes!=np.float64))[0])

In [64]:
columnTransformer=ColumnTransformer([('encoder',OneHotEncoder(),string_columns)],remainder='passthrough')


In [65]:
x=columnTransformer.fit_transform(x)

In [66]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=1)

In [67]:
sc=preprocessing.StandardScaler(with_mean=False)

In [68]:
sc.fit(x_train)

StandardScaler(with_mean=False)

In [69]:
x_train=sc.transform(x_train)


In [70]:
sc.fit(x_test)

StandardScaler(with_mean=False)

In [71]:
x_test=sc.transform(x_test)

In [72]:

#linear regression
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
model=LinearRegression()
model.fit(x_train,y_train)
y_pred=model.predict(x_test)
r2_score(y_test,y_pred)

0.7348974113377778

In [73]:
#polynomial regression

In [74]:
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
model=LinearRegression()
poly=PolynomialFeatures(degree=2)
x_train_poly=poly.fit_transform(x_train)
model.fit(x_train_poly,y_train)
x_test_poly=poly.fit_transform(x_test)
y_pred=model.predict(x_test_poly)
r2_score(y_test,y_pred)

0.40091645871944503

In [75]:
#decision tree model

In [76]:
from sklearn.tree import DecisionTreeRegressor
model=DecisionTreeRegressor(max_depth=4)
model.fit(x_train,y_train)
y_pred=model.predict(x_test)
r2_score(y_test,y_pred)

-0.009129723533083611

In [77]:
#random forest model

In [78]:
from sklearn.ensemble import RandomForestRegressor
model=RandomForestRegressor(n_estimators=25,random_state=10)
model.fit(x_train,y_train)
y_pred=model.predict(x_test)
r2_score(y_test,y_pred)

0.790815348701032

In [79]:
from sklearn.ensemble  import AdaBoostRegressor
model=AdaBoostRegressor(random_state=0,n_estimators=100)
model.fit(x_train,y_train)
y_pred=model.predict(x_test)
r2_score(y_test,y_pred)

0.6954381734250454

In [80]:
#XG BOOST 

In [81]:
pip install xgboost

Note: you may need to restart the kernel to use updated packages.


In [82]:
import xgboost as xgb
model=xgb.XGBRegressor(
    n_estimators=100,
    reg_lambda=1,
    gamma=0,
    max_depth=3,
    learning_rate=0.05   
)
model.fit(x_train,y_train)
y_pred=model.predict(x_test)
r2_score(y_test,y_pred)

0.7770638165984833

In [83]:
from sklearn.svm import SVR
model=SVR()
model.fit(x_train,y_train)
y_pred=model.predict(x_test)
r2_score(y_test,y_pred)

-0.027112906548981686

In [84]:
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeRegressor
model=DecisionTreeRegressor()

params={'max_leaf_nodes':list(range(2,100)),'min_samples_split':[2,3,4],
       'random_state':[42]}

grid_search_cv=GridSearchCV(estimator=model,
                           param_grid=params,verbose=1,cv=10,scoring='r2',n_jobs=-1)
grid_search_cv.fit(x_train,y_train)
print("Best Estimator",grid_search_cv.best_estimator_)
print("Best Score",grid_search_cv.best_score_)

Fitting 10 folds for each of 294 candidates, totalling 2940 fits
Best Estimator DecisionTreeRegressor(max_leaf_nodes=30, min_samples_split=3, random_state=42)
Best Score 0.7470155193420238


In [87]:
#grid search cv for random forest tree model
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor
model=RandomForestRegressor(random_state=42)
params={
'bootstrap':[True],
'max_depth':[80,90,100,110],
'max_features':[2,3],
'min_samples_leaf':[3,4,5],
'min_samples_split':[8,10,12],
'n_estimators':[100,200,300,1000]
}
grid_search_cv=GridSearchCV(estimator=model,
                           param_grid=params,verbose=1,cv=10,scoring='r2',n_jobs=-1)

grid_search_cv.fit(x_train,y_train)
print("Best estimator",grid_search_cv.best_estimator_)
print("Best score",grid_search_cv.best_score_)

Fitting 10 folds for each of 288 candidates, totalling 2880 fits
Best estimator RandomForestRegressor(max_depth=80, max_features=3, min_samples_leaf=3,
                      min_samples_split=8, random_state=42)
Best score 0.05808915150276002
